In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import random
import seaborn as sn
import datetime as dt
from spacy.lang.en.stop_words import STOP_WORDS
import nltk
nltk.download('stopwords')
from keras import backend as K
import re
from sklearn.utils import resample
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from gensim.models import Word2Vec, KeyedVectors
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, SCORERS, precision_recall_curve, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import Lasso, LassoLars, LinearRegression, Ridge
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 2.8MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30315 sha256=9cee698cf3674a454d71f09daa578239efb3fd7e804dfa99659caa17ed5b799c
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=6038852ad4af19f162bb3cc09559202e17638570c9641af8a2ac7286dae53fdd
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=eb9cd996266c2ff3ed17ddb075fe196b3c2442d68c14c7a2ec673465161d50eb
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 6.4MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from keras import layers, models, optimizers, regularizers
import bert

In [ ]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/My Drive/independence/data.csv')

In [ ]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
data.head()

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,new_text,title_abstract
0,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1.0,0.0,0.0,0.0,0.0,0.0,reconstructing subject specific effect maps pr...,Reconstructing Subject-Specific Effect Maps P...
1,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1.0,0.0,0.0,0.0,0.0,0.0,rotation invariance neural network rotation in...,Rotation Invariance Neural Network Rotation i...
2,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0.0,0.0,1.0,0.0,0.0,0.0,spherical polyharmonics and poisson kernels fo...,Spherical polyharmonics and Poisson kernels fo...
3,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0.0,0.0,1.0,0.0,0.0,0.0,finite element approximation for the stochasti...,A finite element approximation for the stochas...
4,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1.0,0.0,0.0,1.0,0.0,0.0,comparative study discrete wavelet transforms ...,Comparative study of Discrete Wavelet Transfor...


In [ ]:
data['title_abstract'] = data.TITLE + data.ABSTRACT

In [ ]:
pat3 = re.compile(r'[_,-]')
pat = re.compile(r'\w{3,}')
pat4 = re.compile('\D+')

In [ ]:
stop_words = list(STOP_WORDS)
stop_words.extend(stopwords.words('english'))

In [ ]:
data['new_text'] = data['new_text'].map(lambda x:pat3.sub("",x))
data['new_text'] = data['new_text'].map(lambda x:' '.join(pat.findall(x)))
data['new_text'] = data['new_text'].map(lambda x:''.join(pat4.findall(x)))

In [ ]:
data['title_abstract'] = data['title_abstract'].map(lambda x: re.sub(r'[^A-Za-z]', ' ', x).lower())
data['title_abstract'] = data['title_abstract'].map(lambda x: re.sub(r" +", ' ', x))
data.head()

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,new_text,title_abstract
0,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1.0,0.0,0.0,0.0,0.0,0.0,reconstructing subject specific effect maps pr...,reconstructing subject specific effect maps pr...
1,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1.0,0.0,0.0,0.0,0.0,0.0,rotation invariance neural network rotation in...,rotation invariance neural network rotation in...
2,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0.0,0.0,1.0,0.0,0.0,0.0,spherical polyharmonics and poisson kernels fo...,spherical polyharmonics and poisson kernels fo...
3,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0.0,0.0,1.0,0.0,0.0,0.0,finite element approximation for the stochasti...,a finite element approximation for the stochas...
4,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1.0,0.0,0.0,1.0,0.0,0.0,comparative study discrete wavelet transforms ...,comparative study of discrete wavelet transfor...


In [ ]:
target = data.iloc[:, 2:8].dropna()

In [ ]:
print(target['Computer Science'].value_counts(),'\n')
print(target['Physics'].value_counts(), '\n') 
print(target['Mathematics'].value_counts(), '\n') 
print(target['Statistics'].value_counts(), '\n')
print(target['Quantitative Biology'].value_counts(), '\n')
print(target['Quantitative Finance'].value_counts(), '\n')

0.0    12378
1.0     8594
Name: Computer Science, dtype: int64 

0.0    14959
1.0     6013
Name: Physics, dtype: int64 

0.0    15354
1.0     5618
Name: Mathematics, dtype: int64 

0.0    15766
1.0     5206
Name: Statistics, dtype: int64 

0.0    20385
1.0      587
Name: Quantitative Biology, dtype: int64 

0.0    20723
1.0      249
Name: Quantitative Finance, dtype: int64 



In [ ]:
fulltokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2', trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
tokenizer = fulltokenizer(vocab_file)

In [ ]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("my name is going to be sparsh"))

[2026, 2171, 2003, 2183, 2000, 2022, 12403, 2869, 2232]

In [ ]:
def encode_sentence(sent):
  return ["[CLS]"]+tokenizer.tokenize(sent)+["[SEP]"]

In [ ]:
text_input = [encode_sentence(sentence) for sentence in data.title_abstract]

In [ ]:
def get_ids(tokens):
  return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
  return np.char.not_equal(tokens, "[PAD]").astype('int')  

def get_segments(tokens):
  seg_ids = []
  current_seg_id = 0
  for tok in tokens:
    seg_ids.append(current_seg_id)
    if tok == "[SEP]":
      current_seg_id = 1-current_seg_id  
  return seg_ids  

In [ ]:
data_with_len = [[sent, target['Computer Science'][i], len(sent)]
                 for i, sent in enumerate(text_input[:20972])]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
               sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]
      

In [ ]:
data_with_len_test = [[sent, len(sent)]
                 for i, sent in enumerate(text_input[20972:])]

sorted_all_test = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
               sent_lab[1])
              for sent_lab in data_with_len_test]        

In [ ]:
sorted_all[0][0][0]

[101,
 2784,
 15756,
 6125,
 1037,
 4766,
 2381,
 4955,
 2000,
 2784,
 15756,
 6125,
 1998,
 2037,
 2381,
 102]

In [ ]:
# A list is a type of iterator so it can be used as generator for a dataset
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [ ]:
NB_BATCHES = np.math.ceil(len(sorted_all) / BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
val_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [ ]:
next(iter(val_dataset))

(<tf.Tensor: shape=(32, 3, 29), dtype=int32, numpy=
 array([[[  101,  2784, 15756, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  5110,  2522, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101, 15182,  1999, ...,     0,     0,     0],
         [    1,     1,     1, ...,     0,     0,     0],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        ...,
 
        [[  101,  1996,  2832, ...,  4730,  2653,   102],
         [    1,     1,     1, ...,     1,     1,     1],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101, 10124,  9972, ..., 10124,  9972,   102],
         [    1,     1,     1, ...,     1,     1,     1],
         [    0,     0,     0, ...,     0,     0,     0]],
 
        [[  101,  1996, 20248, ...,  2389, 19287

## Model Building

In [ ]:
class DCNNBERTEmbedding(tf.keras.Model):
    
    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=False)
        
        self.unigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=1,
                                    padding="valid",
                                    activation="relu")

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.5, l2=0.5))
        self.dropout = layers.Dropout(rate=dropout_rate)
        self.dense_1 = layers.Dense(units=FFN_units*2, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.5, l2=0.5))
        self.dropout = layers.Dropout(rate=dropout_rate)
        self.dense_1 = layers.Dense(units=FFN_units*3, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.5, l2=0.5))
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def embed_with_bert(self, all_tokens):
        _, embs = self.bert_layer([all_tokens[:, 0, :],
                                   all_tokens[:, 1, :],
                                   all_tokens[:, 2, :]])
        return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)

        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

In [ ]:
NB_FILTERS = 100
FFN_UNITS = 128
NB_CLASSES = 2

DROPOUT_RATE = 0.5

BATCH_SIZE = 32
NB_EPOCHS = 5

In [ ]:
def f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="rmsprop",
                 metrics=["accuracy", f1])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [ ]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS, validation_data=val_dataset)

Epoch 1/5
591/591 [==============================] - 535s 905ms/step - loss: 145.6824 - accuracy: 0.6966 - f1: 0.5860 - val_loss: 29.7431 - val_accuracy: 0.7726 - val_f1: 0.5497
Epoch 2/5
591/591 [==============================] - 535s 905ms/step - loss: 29.5083 - accuracy: 0.7021 - f1: 0.5393 - val_loss: 29.1003 - val_accuracy: 0.8683 - val_f1: 0.5686
Epoch 3/5
591/591 [==============================] - 534s 904ms/step - loss: 29.4886 - accuracy: 0.7288 - f1: 0.5743 - val_loss: 29.3193 - val_accuracy: 0.8817 - val_f1: 0.6750
Epoch 4/5
591/591 [==============================] - 534s 904ms/step - loss: 29.4213 - accuracy: 0.7831 - f1: 0.6968 - val_loss: 29.4321 - val_accuracy: 0.8466 - val_f1: 0.6821
Epoch 5/5
591/591 [==============================] - 534s 903ms/step - loss: 29.3852 - accuracy: 0.8058 - f1: 0.7536 - val_loss: 29.4017 - val_accuracy: 0.8740 - val_f1: 0.7009


In [ ]:
Dcnn.evaluate(test_dataset)

65/65 [==============================] - 20s 305ms/step - loss: 29.4017 - accuracy: 0.8740 - f1: 0.7009


[29.401681900024414, 0.8740384578704834, 0.7009085416793823]

In [ ]:
pred_qb = dcnn.predict(seq[20972:])

In [ ]:
pred_qb[pred_qb<0.5] = 0  
pred_qb[pred_qb>=0.5] = 1  

In [ ]:
pred_qb.sum()

63.0